In [ ]:
import os
import numpy as np 
import sys

sys.path.append("..")
import tomondt
from tomondt import Operator
import pickle
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import mrcz

main_path = 'Y:\TimC\DIPSTER-PublicationData\Publication-Data'
v_path = os.path.join(main_path, 'Reconstructions')
ts_path = os.path.join(main_path, 'Volumes')
view = tomondt.Viewer()

ref = tomondt.load_vmf(os.path.join(ts_path,'Cage-1.vmf')).read_record(0).astype(np.float64)
ts_path_2 = os.path.join(main_path, 'TiltSeries')
ts = pickle.load(open(os.path.join(ts_path_2,'Cage-1-ts.pkl'), 'rb'))
angles = ts.angles
ts.data = tomondt.algorithms.forward_astra(ref,angles).astype(np.float64)

ts.data = tomondt.utils.permute(ts.data,(0,1,2))
vol =  tomondt.algorithms.sirt_astra(ts,100)
print(ssim(ref,vol,data_range=1.1))

mrcz.writeMRC(vol, os.path.join(v_path,'Cage-1-ts-70-rec.rec'))
view.add_image(vol)
view.add_image(ref)

In [2]:
import os
import numpy as np 
import sys

sys.path.append("..")
import tomondt
from tomondt import Operator
import pickle
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import mrcz
from dipster import tomo

main_path = 'Y:\TimC\DIPSTER-PublicationData\Publication-Data'
v_path = os.path.join(main_path, 'Reconstructions')
ts_path = os.path.join(main_path, 'Volumes')
view = tomondt.Viewer()

ref = tomondt.load_vmf(os.path.join(ts_path,'Cage-1.vmf')).read_record(0).astype(np.float64)
ts_path_2 = os.path.join(main_path, 'TiltSeries')
ts = pickle.load(open(os.path.join(ts_path_2,'Cage-1-ts.pkl'), 'rb'))
ts.data = ts.data[:,:,:,None]
angles = ts.angles
for i in range(ref.shape[2]):
    ts.data[:,i,:,:][:,None,:,:] = tomo.fp_astra(ref[:,i,:,None][:,None,:,:],angles).astype(np.float64)
view.add_image(ts.data.squeeze())
#arr = np.tile(angles, (len(angles), 1)) 
#vol = np.zeros((512,512,512,1))
#for i in range(ref.shape[1]):
#    vol[:,i,:,:][:,None,:,:] =  tomo.bp_astra(ts.data[:,i,:,:][:,None,:,:],arr,20)
#print(ssim(ref,vol.squeeze(),data_range=1.1))

#mrcz.writeMRC(vol, os.path.join(v_path,'Cage-1-ts-70-rec.rec'))
#view.add_image(vol.squeeze())
#view.add_image(ref)

<Image layer 'Image' at 0x1a2b0d01010>

In [ ]:
obj = tomondt.utils.permute(vol,(0,1,2,3)).squeeze()
print(ssim(ref,obj,data_range=1.1))
view = tomondt.Viewer()
view.add_image(obj)
view.add_image(ref)